In [5]:
import pandas as pd

In [4]:
import pandas as pd
df = pd.read_csv('spotify_millsongdata.csv', on_bad_lines='skip', engine='python') # Use the Python engine for parsing
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [6]:
df.shape

(57650, 4)

In [7]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [8]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)
df.head(10)

,artist,song,text
0,ABBA,Reina Danzante,You can dance \r\nYou can jive \r\nHaving th...
1,Queen,Don't Try So Hard,If you're searching out for something \r\nDon...
2,Hillsong,Freedom is here,The future comes alive \r\nYou speak Your wor...
3,You Am I,Tuesday,Saw the sun come up again. \r\nLast week's pa...
4,Rush,Before And After,Love won't see my coming \r\nOn a Sunday noon...
5,Underoath,We Are The Involuntary,Just drifting along with the world \r\nEvery ...
6,Dan Fogelberg,Once Upon A Time,Every morning you wake up alone \r\nAnd you s...
7,Quietdrive,Rise From The Ashes,I felt this way before \r\nI fell right throu...
8,Def Leppard,Pour Some Sugar On Me,"Hey, hey, hey \r\n \r\nLove is like a bomb b..."
9,Virgin Steele,Last Supper,Father forgive them for they know not what the...


In [9]:
df['text'][0]

"You can dance  \r\nYou can jive  \r\nHaving the time of your life  \r\nSee that girl  \r\nWatch that scene  \r\nDig in the dancing queen  \r\n  \r\nFriday night and the lights are low  \r\nLooking out for a place to go  \r\nWhere they play the right music  \r\nGetting in the swing  \r\nYou come to look for a king  \r\nAnybody could be that guy  \r\nNight is young and the music's high  \r\nWith a bit of rock music  \r\nEverything is fine  \r\nYou're in the mood for a dance  \r\nAnd when you get the chance  \r\n  \r\nYou are the dancing queen  \r\nYoung and sweet  \r\nOnly seventeen  \r\nDancing queen  \r\nFeel the beat from the tambourine  \r\nYou can dance  \r\nYou can jive  \r\nHaving the time of your life  \r\nSee that girl  \r\nWatch that scene  \r\nDig in the dancing queen  \r\n  \r\nYou're a teaser, you turn 'em on  \r\nLeave 'em burning and then you're gone  \r\nLooking out for another  \r\nAnyone will do  \r\nYou're in the mood for a dance  \r\nAnd when you get the chance  \r\n

In [10]:
#text cleaning/text processing
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex=True)

In [11]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [13]:
import nltk
# Download the 'punkt' resource
nltk.download('punkt')

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    # Check if txt is a string before tokenizing
    if isinstance(txt, str):
        tokens = nltk.word_tokenize(txt)
        stemming = [stemmer.stem(w) for w in tokens]
        return " ".join(stemming)
    else:
        # Handle non-string values (e.g., return an empty string or None)
        return ""

df['text'] = df['text'].apply(lambda x: tokenization(x))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [15]:
tfidvector=TfidfVectorizer(analyzer='word',stop_words='english')
matrix=tfidvector.fit_transform(df['text'])
similarity=cosine_similarity(matrix)

In [16]:
similarity[0]

array([1.        , 0.0215441 , 0.02929959, ..., 0.00149773, 0.05018644,
       0.01972225])

In [18]:
df[df['song'] == 'Crying Over You']

,artist,song,text
1980,UB40,Crying Over You,cri over you in the morn cri over you in the e...


In [19]:
def recommendation(song_df):
    # Check if the song exists in the dataframe
    if df[df['song'] == song_df].shape[0] == 0:
        return "Song not found!"

    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])

    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)

    return songs

In [20]:
recommendation('For Your Pleasure')

'Song not found!'

In [21]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))


Mounted at /content/drive
